In [94]:
%pip install simpy
print('Versión:', simpy.__version__)

Versión: 4.1.1


In [95]:
### Un auto estaciona durante 5 minutos y luego maneja durante 2 minutos.

### Simular durante 2 horas

def auto(env):
  while True:
    print('Estacionando a las %d' % env.now)
    tiempoEstacionado = 5
    yield env.timeout(tiempoEstacionado)
    print('Manejando a las %d' % env.now)
    tiempoViaje = 2
    yield env.timeout(tiempoViaje)

In [96]:
import simpy
env = simpy.Environment()
env.process(auto(env))
env.run(until=120)

Estacionando a las 0
Manejando a las 5
Estacionando a las 7
Manejando a las 12
Estacionando a las 14
Manejando a las 19
Estacionando a las 21
Manejando a las 26
Estacionando a las 28
Manejando a las 33
Estacionando a las 35
Manejando a las 40
Estacionando a las 42
Manejando a las 47
Estacionando a las 49
Manejando a las 54
Estacionando a las 56
Manejando a las 61
Estacionando a las 63
Manejando a las 68
Estacionando a las 70
Manejando a las 75
Estacionando a las 77
Manejando a las 82
Estacionando a las 84
Manejando a las 89
Estacionando a las 91
Manejando a las 96
Estacionando a las 98
Manejando a las 103
Estacionando a las 105
Manejando a las 110
Estacionando a las 112
Manejando a las 117
Estacionando a las 119


In [97]:
### Un persona llega al sistema cada 5+/-1 minutos, realiza una tarea durante 4+3 minutos y luego se va.

### Simular durante una hora

In [98]:
import simpy
import random

personasQueSeFueron = 0

def persona(env, nro):
    global personasQueSeFueron
    print(f"Ingresó la persona {nro} a las {env.now}")
    yield env.timeout(random.randint(1, 7))
    print(f"Se fue la persona {nro} a las {env.now}")
    personasQueSeFueron+=1

def sistema(env):
    i=0
    while True:
      yield env.timeout(random.randint(4, 6))
      i+=1
      env.process(persona(env, i))

env = simpy.Environment()
env.process(sistema(env))
env.run(until=60)
print(f"Se fueron del sistema {personasQueSeFueron} personas")

Ingresó la persona 1 a las 5
Se fue la persona 1 a las 7
Ingresó la persona 2 a las 11
Ingresó la persona 3 a las 15
Se fue la persona 2 a las 16
Ingresó la persona 4 a las 19
Se fue la persona 3 a las 19
Se fue la persona 4 a las 23
Ingresó la persona 5 a las 25
Se fue la persona 5 a las 26
Ingresó la persona 6 a las 31
Se fue la persona 6 a las 34
Ingresó la persona 7 a las 37
Ingresó la persona 8 a las 41
Se fue la persona 7 a las 43
Ingresó la persona 9 a las 46
Se fue la persona 8 a las 47
Se fue la persona 9 a las 48
Ingresó la persona 10 a las 51
Se fue la persona 10 a las 56
Ingresó la persona 11 a las 57
Se fueron del sistema 10 personas


In [99]:
# ¿Y si quisiera finalizar la simulación cuando se hayan retirado 20 personas del sistema?

# Alternativa 1: lanzando una excepción

import simpy
import random
from simpy.core import StopSimulation

personasQueSeFueron = 0
LIMITE_PERSONAS = 20

def persona(env, nro):
    global personasQueSeFueron
    print(f"Ingresó la persona {nro} a las {env.now}")
    yield env.timeout(random.randint(1, 7))
    print(f"Se fue la persona {nro} a las {env.now}")
    personasQueSeFueron+=1
    if personasQueSeFueron >= LIMITE_PERSONAS:
        print(f"Se alcanzó el límite de {LIMITE_PERSONAS} personas. Finalizando simulación.")
        raise StopSimulation("Fin normal de la simulación")

def sistema(env):
    i=0
    while True:
      yield env.timeout(random.randint(4, 6))
      i+=1
      env.process(persona(env, i))

env = simpy.Environment()
env.process(sistema(env))
env.run()

Ingresó la persona 1 a las 6
Se fue la persona 1 a las 7
Ingresó la persona 2 a las 10
Se fue la persona 2 a las 13
Ingresó la persona 3 a las 16
Se fue la persona 3 a las 20
Ingresó la persona 4 a las 22
Se fue la persona 4 a las 24
Ingresó la persona 5 a las 28
Se fue la persona 5 a las 32
Ingresó la persona 6 a las 34
Se fue la persona 6 a las 35
Ingresó la persona 7 a las 40
Ingresó la persona 8 a las 44
Se fue la persona 7 a las 45
Ingresó la persona 9 a las 48
Se fue la persona 8 a las 50
Se fue la persona 9 a las 51
Ingresó la persona 10 a las 54
Se fue la persona 10 a las 58
Ingresó la persona 11 a las 59
Ingresó la persona 12 a las 63
Se fue la persona 12 a las 65
Se fue la persona 11 a las 66
Ingresó la persona 13 a las 69
Ingresó la persona 14 a las 75
Se fue la persona 13 a las 75
Ingresó la persona 15 a las 80
Se fue la persona 14 a las 80
Se fue la persona 15 a las 84
Ingresó la persona 16 a las 85
Se fue la persona 16 a las 87
Ingresó la persona 17 a las 91
Se fue la per

'Fin normal de la simulación'

In [100]:
# Alternativa 2: disparando un evento

import simpy
import random

personasQueSeFueron = 0
LIMITE_PERSONAS = 20

def persona(env, nro, evento_fin):
    global personasQueSeFueron
    print(f"Ingresó la persona {nro} a las {env.now}")
    yield env.timeout(random.randint(1, 7))
    print(f"Se fue la persona {nro} a las {env.now}")
    personasQueSeFueron += 1

    if personasQueSeFueron >= LIMITE_PERSONAS and not evento_fin.triggered:
        print(f"Se alcanzó el límite de {LIMITE_PERSONAS} personas. Finalizando simulación.")
        evento_fin.succeed("Fin normal de la simulación")  # se dispara el evento

def sistema(env, evento_fin):
    i = 0
    while True:
        yield env.timeout(random.randint(4, 6))
        i += 1
        env.process(persona(env, i, evento_fin))

env = simpy.Environment()
evento_fin = env.event()  # este evento disparará más adelante cuando se retiren 20 personas
env.process(sistema(env, evento_fin))
resultado = env.run(until=evento_fin)  # la simulación terminará cuando el evento se dispare

print("Simulación finalizada.")
print(f"Motivo: {resultado}")

Ingresó la persona 1 a las 5
Ingresó la persona 2 a las 10
Se fue la persona 1 a las 11
Se fue la persona 2 a las 15
Ingresó la persona 3 a las 16
Ingresó la persona 4 a las 22
Se fue la persona 3 a las 23
Se fue la persona 4 a las 23
Ingresó la persona 5 a las 27
Se fue la persona 5 a las 29
Ingresó la persona 6 a las 33
Se fue la persona 6 a las 36
Ingresó la persona 7 a las 37
Se fue la persona 7 a las 38
Ingresó la persona 8 a las 41
Se fue la persona 8 a las 44
Ingresó la persona 9 a las 46
Se fue la persona 9 a las 49
Ingresó la persona 10 a las 50
Se fue la persona 10 a las 51
Ingresó la persona 11 a las 55
Se fue la persona 11 a las 59
Ingresó la persona 12 a las 60
Ingresó la persona 13 a las 65
Se fue la persona 12 a las 66
Se fue la persona 13 a las 67
Ingresó la persona 14 a las 71
Se fue la persona 14 a las 74
Ingresó la persona 15 a las 75
Ingresó la persona 16 a las 80
Se fue la persona 15 a las 82
Ingresó la persona 17 a las 84
Se fue la persona 16 a las 84
Ingresó la p

In [101]:
# Vamos a resolver el ejercicio del auto utilizando objetos

## El auto del primer ejercicio, en vez de estacionar un cierto tiempo, estaciona y carga bateria
## Recien cuando la bateria está cargada, puede comenzar a andar durante 2 minutos.
## La bateria demora 5 minutos en cargarse.

## Simular durante una hora

import simpy

class Auto(object):

  def __init__(self, env):
    self.env = env
    self.action = env.process(self.arranca())

  def arranca(self):
      while True:
          print(f'Estacionando y cargando energía a las {self.env.now}')
          duracionCarga = 5
          yield self.env.process(self.cargaBateria(duracionCarga))
          print(f'Comienza a andar a las {self.env.now}')
          yield self.env.timeout(2)

  def cargaBateria(self, duracionCarga):
              yield self.env.timeout(duracionCarga)

env = simpy.Environment()
auto = Auto(env)
env.run(until=60)

Estacionando y cargando energía a las 0
Comienza a andar a las 5
Estacionando y cargando energía a las 7
Comienza a andar a las 12
Estacionando y cargando energía a las 14
Comienza a andar a las 19
Estacionando y cargando energía a las 21
Comienza a andar a las 26
Estacionando y cargando energía a las 28
Comienza a andar a las 33
Estacionando y cargando energía a las 35
Comienza a andar a las 40
Estacionando y cargando energía a las 42
Comienza a andar a las 47
Estacionando y cargando energía a las 49
Comienza a andar a las 54
Estacionando y cargando energía a las 56


In [102]:
# Agregamos interrupciones

# Al mismo auto que está estacionado cargando bateria, aparece el
# conductor a los 3 minutos e interrumpe la carga para comenzar a manejar

# Simular durante 25 minutos

def conductor(env, auto):
  yield env.timeout(3)
  auto.action.interrupt()

class Auto(object):

  def __init__(self, env):
    self.env = env
    self.action = env.process(self.arranca())

  def arranca(self):
      while True:
          print(f'Estacionando y cargando energía a las {self.env.now}')
          duracionCarga = 5

          try:
            yield self.env.process(self.cargaBateria(duracionCarga))
          except simpy.Interrupt:
            print(f'Se interrumpió la carga a las {self.env.now}. Esperemos que alcance para el viaje...')

          print(f'Comienza a andar a las {self.env.now}')
          yield self.env.timeout(2)

  def cargaBateria(self, duracionCarga):
              yield self.env.timeout(duracionCarga)

import simpy
env = simpy.Environment()
auto = Auto(env)
env.process(conductor(env, auto))

env.run(until=25)

Estacionando y cargando energía a las 0
Se interrumpió la carga a las 3. Esperemos que alcance para el viaje...
Comienza a andar a las 3
Estacionando y cargando energía a las 5
Comienza a andar a las 10
Estacionando y cargando energía a las 12
Comienza a andar a las 17
Estacionando y cargando energía a las 19
Comienza a andar a las 24


In [108]:
# Agregamos un recurso compartido (similar al storage)

# Se desea simular el funcionamiento de un puesto de carga para autos eléctricos.
#Este puesto permite que un máximo de 2 autos carguen sus baterías en simultáneo.
# Al inicio de la simulación, se presentan 15 autos eléctricos que desean realizar la carga.
# Cada auto sigue la siguiente lógica:
### Recorre el camino hacia el puesto de carga (demora: 2 minutos).
### Espera en una cola si el puesto está ocupado.
### Carga su batería durante 5 minutos cuando accede al recurso.
### Luego de la carga, abandona el sistema.

# Simular durante 30 minutos

import simpy

def auto(env, nombre, puestoBaterias, tiempoConduccion, tiempoCarga):

  print(f'El auto {nombre} comienza a conducir a las {env.now}')
  yield env.timeout(tiempoConduccion)
  print(f'El auto {nombre} frena en el puesto de baterias a las {env.now}')

  with puestoBaterias.request() as req:
    yield req
    print (f'El auto {nombre} comienza a cargar bateria a las {env.now}')
    yield env.timeout(tiempoCarga)
    print (f'El auto {nombre} finalizó su carga bateria a las {env.now}')

# Importante:
# Si el recurso se toma con un with, se libera en forma automática luego de finalizado el with
# Si el recurso no se toma con un with, se debe liberar mediante el release() una vez que se terminó de utilizar

import simpy
env = simpy.Environment()
puestoBaterias = simpy.Resource(env, capacity=2)
for i in range(1,16):
  env.process(auto(env, i, puestoBaterias, 2, 5))

env.run(until=30)

El auto 1 comienza a conducir a las 0
El auto 2 comienza a conducir a las 0
El auto 3 comienza a conducir a las 0
El auto 4 comienza a conducir a las 0
El auto 5 comienza a conducir a las 0
El auto 6 comienza a conducir a las 0
El auto 7 comienza a conducir a las 0
El auto 8 comienza a conducir a las 0
El auto 9 comienza a conducir a las 0
El auto 10 comienza a conducir a las 0
El auto 11 comienza a conducir a las 0
El auto 12 comienza a conducir a las 0
El auto 13 comienza a conducir a las 0
El auto 14 comienza a conducir a las 0
El auto 15 comienza a conducir a las 0
El auto 1 frena en el puesto de baterias a las 2
El auto 2 frena en el puesto de baterias a las 2
El auto 3 frena en el puesto de baterias a las 2
El auto 4 frena en el puesto de baterias a las 2
El auto 5 frena en el puesto de baterias a las 2
El auto 6 frena en el puesto de baterias a las 2
El auto 7 frena en el puesto de baterias a las 2
El auto 8 frena en el puesto de baterias a las 2
El auto 9 frena en el puesto de 

In [109]:
# ¿y si quisieramos finalizar la simulación cuando todos los autos hayan terminado de
# cargar su batería? ¿En qué instante finaliza la simulación?

import simpy

def auto(env, nombre, puestoBaterias, tiempoConduccion, tiempoCarga):

  print(f'El auto {nombre} comienza a conducir a las {env.now}')
  yield env.timeout(tiempoConduccion)
  print(f'El auto {nombre} frena en el puesto de baterias a las {env.now}')

  with puestoBaterias.request() as req:
    yield req
    print (f'El auto {nombre} comienza a cargar bateria a las {env.now}')
    yield env.timeout(tiempoCarga)
    print (f'El auto {nombre} finalizó su carga bateria a las {env.now}')

#Si se toma el recurso con un with, se libera en forma automática luego de finalizado el with
#Si no se toma con un with, se debe liberar mediante el release() una vez que se terminó de utilizar

import simpy
env = simpy.Environment()
puestoBaterias = simpy.Resource(env, capacity=2)
for i in range(1,16):
  env.process(auto(env, i, puestoBaterias, 2, 5))

env.run()   # La simulación finaliza cuando ya no existan más eventos

El auto 1 comienza a conducir a las 0
El auto 2 comienza a conducir a las 0
El auto 3 comienza a conducir a las 0
El auto 4 comienza a conducir a las 0
El auto 5 comienza a conducir a las 0
El auto 6 comienza a conducir a las 0
El auto 7 comienza a conducir a las 0
El auto 8 comienza a conducir a las 0
El auto 9 comienza a conducir a las 0
El auto 10 comienza a conducir a las 0
El auto 11 comienza a conducir a las 0
El auto 12 comienza a conducir a las 0
El auto 13 comienza a conducir a las 0
El auto 14 comienza a conducir a las 0
El auto 15 comienza a conducir a las 0
El auto 1 frena en el puesto de baterias a las 2
El auto 2 frena en el puesto de baterias a las 2
El auto 3 frena en el puesto de baterias a las 2
El auto 4 frena en el puesto de baterias a las 2
El auto 5 frena en el puesto de baterias a las 2
El auto 6 frena en el puesto de baterias a las 2
El auto 7 frena en el puesto de baterias a las 2
El auto 8 frena en el puesto de baterias a las 2
El auto 9 frena en el puesto de 

# Ejercicio 1: Máquina de café con uso exclusivo

Las personas llegan a una máquina de café con un tiempo entre llegadas generado por una **exponencial negativa con media 4 minutos** (`random.expovariate(1/4)`).

Cada persona tarda **1.5 minutos** en servirse café.

Simular durante **60 minutos**.

Mostrar cuántas personas llegaron, cuántas fueron atendidas y cuántas quedaron esperando.


# Ejercicio 2: Reparación de celulares

En un local de reparación hay **2 técnicos**.

Las personas llegan cada 6 minutos.

El tiempo de reparación es aleatorio: **puede ser 4 o 6 minutos con igual probabilidad** (`random.choice([4, 6])`).

Simular durante **1 hora**.

Mostrar cuándo cada cliente fue atendido y se retiró.


# Ejercicio 3: Llegada de colectivos a una dársena

Colectivos llegan a una terminal cada **7 o 9 minutos** (elegido aleatoriamente con igual probabilidad).

Cada colectivo ocupa la dársena durante **6 minutos**.

Simular hasta que lleguen **10 colectivos**.

Mostrar los tiempos de llegada, inicio de carga y salida de cada uno.


# Ejercicio 4: Autos en túnel de lavado

Un túnel de lavado puede atender hasta **3 autos al mismo tiempo**.

Llegan **30 autos** al sistema, uno cada 2 minutos.

Cada lavado dura entre **4 y 6 minutos** (`random.uniform(4, 6)`).

Finalizar la simulación cuando todos los autos hayan sido atendidos.

Mostrar los tiempos de inicio y fin del lavado de cada auto.


# Ejercicio 5: Técnico interrumpido por supervisor

Un técnico comienza una calibración que dura **10 minutos**.

A los **4 minutos**, el supervisor interrumpe el trabajo.

El técnico detiene su tarea y se dedica a otra cosa.

Simular **20 minutos**.

Mostrar cuándo ocurre la interrupción y qué hace el técnico luego.


# Ejercicio 6: Reparación de bicicletas

Bicicletas llegan a un taller cada **4 o 6 minutos** (elegido al azar con `random.choice([4, 6])`).

El mecánico trabaja **de a una bicicleta a la vez**, y cada reparación tarda **7 minutos**.

Simular durante **60 minutos**.

Mostrar cuántas bicicletas fueron reparadas.


# Ejercicio 7: Guardia médica con atención variable

Dos médicos atienden pacientes en una guardia.

Los pacientes llegan cada **6 minutos**.

La atención médica dura **8 o 12 minutos**, elegidos al azar con `random.choice([8, 12])`.

Simular durante **90 minutos**.

Mostrar los tiempos de llegada, inicio y finalización de atención de cada paciente.


# Ejercicio 8: Caja única de supermercado

Una caja de supermercado atiende **una persona a la vez**.

Los clientes llegan cada **2 o 4 minutos**, al azar (`random.choice([2, 4])`).

Cada atención dura exactamente **2 minutos**.

Finalizar la simulación luego de atender **25 personas**.

Mostrar tiempos de espera y atención de cada cliente.


# Ejercicio 9: Puesto de carga con evento de finalización

Llegan autos eléctricos cada **5 minutos**.

El puesto de carga tiene **2 cargadores**.

Cada carga dura **5 o 7 minutos**, al azar (`random.choice([5, 7])`).

Cuando se hayan cargado **15 autos**, se dispara un evento que finaliza la simulación.

Mostrar la duración total de la simulación y el momento de carga de cada auto.


# Ejercicio 10: Patrulla interrumpida por una urgencia

Una patrulla realiza rondas que duran **8 minutos**.

A los **5 minutos** de la primera ronda, recibe una **interrupción por una urgencia**.

La patrulla detiene la ronda y acude al evento.

Simular **25 minutos**.

Mostrar cuándo empezó la ronda, cuándo fue interrumpida y qué ocurrió después.
